# Training the model

Imports.

In [160]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

Pathing to the data.

In [161]:
data_path = '../data'
prepared_data_path = f'{data_path}/csv/prepared'

Open the prepared data.

In [162]:
matches_df = pd.read_csv(f'{prepared_data_path}/bundesliga_matches.csv')
matches_df.dtypes

date                                    object
time                                    object
comp                                    object
round                                   object
day                                     object
                                        ...   
shooting_standard_sot_rolling_avg      float64
shooting_standard_dist_rolling_avg     float64
shooting_standard_fk_rolling_avg       float64
shooting_standard_pk_rolling_avg       float64
shooting_standard_pkatt_rolling_avg    float64
Length: 172, dtype: object

Create a method to ease the process of predicting.

In [163]:
def make_predictions(data, predictors):
    """
    Make predictions using the random forest classifier.

    :param data: the dataframe to use
    :param predictors: the predictor columns
    :return: combined: a dataframe containing the actual and predicted values
    """
    # Split the data into train and test sets.
    train_set = data[data['date'] < '2023-02-01']
    test_set = data[data['date'] >= '2023-02-01']
    print(f'Train: {len(train_set)} matches ({len(train_set) / len(data):.2%})')
    print(f'Test: {len(test_set)} matches ({len(test_set) / len(data):.2%})')

    # Create and fit (train) the model.
    model = RandomForestClassifier(
        n_estimators=50,  # number of trees in the forest
        min_samples_split=10,  # number of samples required to split an internal node
        random_state=1  # seed
    )
    model.fit(train_set[predictors], train_set['target'])

    # Make predictions on the test dataset and calculate the precision score.
    predictions = model.predict(test_set[predictors])
    precision = precision_score(test_set['target'], predictions)
    print(f'Precision: {precision:.2%}')

    # Create a dataframe containing the actual and predicted values.
    combined = pd.DataFrame(
        dict(actual=test_set['target'], prediction=predictions),
        index=test_set.index
    )
    return combined

Make predictions on the matches dataframe.

In [164]:
predictors = [
    'opponent_code',
    'venue_code',
    'hour',
    'day_code',
    'gf_rolling_avg',
    'ga_rolling_avg',
    'shooting_standard_sh_rolling_avg',
    'shooting_standard_sot_rolling_avg',
    'shooting_standard_dist_rolling_avg',
    'shooting_standard_fk_rolling_avg',
    'shooting_standard_pk_rolling_avg',
    'shooting_standard_pkatt_rolling_avg'
]
combined = make_predictions(matches_df, predictors)

Train: 3304 matches (91.88%)
Test: 292 matches (8.12%)
Precision: 60.34%


In [165]:
matrix_df = combined.copy()
# Create a mapping dictionary for the replacements.
# We want to make the confusion matrix easier to read.
mapping = {0: 'Loss/Draw', 1: 'Win'}

# Apply the replacements to the 'actual' and 'prediction' columns.
matrix_df['actual'] = matrix_df['actual'].replace(mapping)
matrix_df['prediction'] = matrix_df['prediction'].replace(mapping)

# Create a confusion matrix.
pd.crosstab(index=matrix_df['actual'], columns=matrix_df['prediction'])

prediction,Loss/Draw,Win
actual,,
Loss/Draw,161,23
Win,73,35


Merge to see predictions.

In [166]:
combined = combined.merge(
    matches_df[['date', 'team', 'opponent', 'result']],
    left_index=True,
    right_index=True
)
combined

,actual,prediction,date,team,opponent,result
247,1,0,2023-02-03,Augsburg,Bayer Leverkusen,W
248,0,0,2023-02-11,Augsburg,Mainz 05,L
249,1,0,2023-02-17,Augsburg,Hoffenheim,W
250,0,0,2023-02-25,Augsburg,Hertha BSC,L
251,1,0,2023-03-04,Augsburg,Werder Bremen,W
...,...,...,...,...,...,...
3591,1,0,2023-04-30,Wolfsburg,Mainz 05,W
3592,0,0,2023-05-07,Wolfsburg,Dortmund,L
3593,1,1,2023-05-13,Wolfsburg,Hoffenheim,W
3594,0,0,2023-05-19,Wolfsburg,Freiburg,L


The dataframe above has two rows per match - one containing the prediction for the home team, and one containing the prediction for the away team. We want to combine these two rows into one row per match.

In [167]:
merged = combined.merge(combined, left_on=['date', 'team'], right_on=['date', 'opponent'])
merged

,actual_x,prediction_x,date,team_x,opponent_x,result_x,actual_y,prediction_y,team_y,opponent_y,result_y
0,1,0,2023-02-03,Augsburg,Bayer Leverkusen,W,0,0,Bayer Leverkusen,Augsburg,L
1,0,0,2023-02-11,Augsburg,Mainz 05,L,1,1,Mainz 05,Augsburg,W
2,1,0,2023-02-17,Augsburg,Hoffenheim,W,0,0,Hoffenheim,Augsburg,L
3,0,0,2023-02-25,Augsburg,Hertha BSC,L,1,1,Hertha BSC,Augsburg,W
4,1,0,2023-03-04,Augsburg,Werder Bremen,W,0,0,Werder Bremen,Augsburg,L
...,...,...,...,...,...,...,...,...,...,...,...
283,1,0,2023-04-30,Wolfsburg,Mainz 05,W,0,0,Mainz 05,Wolfsburg,L
284,0,0,2023-05-07,Wolfsburg,Dortmund,L,1,1,Dortmund,Wolfsburg,W
285,1,1,2023-05-13,Wolfsburg,Hoffenheim,W,0,0,Hoffenheim,Wolfsburg,L
286,0,0,2023-05-19,Wolfsburg,Freiburg,L,1,0,Freiburg,Wolfsburg,W


Get the rows where we predicted that **the home team would win and the away team would lose**.

In [168]:
accuracy_df = merged[(merged['prediction_x'] == 1) & (merged['prediction_y'] == 0)]['actual_x'].value_counts()
accuracy = accuracy_df.iloc[0] / (accuracy_df.iloc[0] + accuracy_df.iloc[1]) * 100
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 60.38%
